In [21]:
# import the necessary packages
import numpy as np
import imutils
import cv2
import os
import skimage
import PIL
import matplotlib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from imutils import paths

from sklearn import metrics
#from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [22]:
# grab the list of images that we'll be describing
print("[INFO] describing images...")
imagePaths = list(paths.list_images("."+os.sep+"train_vega"))

# initialize the raw pixel intensities matrix, the features matrix,
# and labels list
rawImages = []
features = []
labels = []
images=[]

[INFO] describing images...


In [23]:
#Function for the images treatment
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    #hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist(image, [0, 1, 2], None, bins,[0, 180, 0, 256, 0, 256])
    
    # handle normalizing the histogram if we are using OpenCV 2.4.X
    if imutils.is_cv2():
        hist = cv2.normalize(hist)

    # otherwise, perform "in place" normalization in OpenCV 3
    else:
        cv2.normalize(hist, hist)
        
    # return the flattened histogram as the feature vector
    return hist.flatten()

#Le number corresponds au nombre d'images qui vont etre générées
#Donc plus il est faible moins longs seront les calculs
def create_distortion(image,number):
    images = []
    for i in range(1,number):
        images.append(np.uint8(image + i*0.5* image.std() * np.random.random(image.shape)))
    return images

def image_treatment(image,label):
    rawImages.append(image_to_feature_vector(image))
    features.append(extract_color_histogram(image))
    labels.append(label)

In [24]:
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
    # load the image and extract the class label (assuming that our
    # path as the format: /path/to/dataset/{class}.{image_num}.jpg
    
    image = cv2.imread(imagePath)
    tab_image = create_distortion(image,11)
    
    image_flip = cv2.flip(image,1)
    tab_image_flip = create_distortion(image_flip,11)
   
    images = tab_image + tab_image_flip
    images.append(image)
    images.append(image_flip)
    
    label = imagePath.split(os.path.sep)[-1].split(".")[0]

    for im in images:
        image_treatment(im,label)

    # show an update every 10 images
    if i > 0 and i % 10 == 0:
        print("[INFO] processed {}/{}".format(i, len(imagePaths)))
print("Done")

[INFO] processed 10/60
[INFO] processed 20/60
[INFO] processed 30/60
[INFO] processed 40/60
[INFO] processed 50/60
Done


In [25]:
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(
    rawImages, labels, test_size=0.25, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(
    features, labels, test_size=0.25, random_state=42)

In [26]:
svm_kernel = ["linear","poly","rbf","sigmoid"]
# train and evaluate a SVM classifer on the raw pixel intensities
print("[INFO] evaluating raw pixel accuracy...")
for k in range(len(svm_kernel)) :
    model = SVC(kernel=svm_kernel[k], C=1, random_state=0)
    model.fit(trainRI, trainRL)
    RL_pred = model.predict(testRI)
    acc = model.score(testRI, testRL)
    print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))
    print(RL_pred)

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 81.52%
['bouche' 'Ankh' 'hibou' 'Ankh' 'hibou' 'hibou' 'hibou' 'Ankh' 'oeil'
 'Ankh' 'hibou' 'bouche' 'hibou' 'lapin' 'bouche' 'Ankh' 'Ankh' 'hibou'
 'Ankh' 'Ankh' 'lapin' 'bouche' 'Ankh' 'bouche' 'hibou' 'hibou' 'Ankh'
 'Ankh' 'lapin' 'Ankh' 'Ankh' 'bouche' 'Ankh' 'lapin' 'oeil' 'Ankh'
 'bouche' 'bouche' 'hibou' 'hibou' 'hibou' 'Ankh' 'hibou' 'hibou' 'Ankh'
 'hibou' 'hibou' 'bouche' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'hibou' 'oeil' 'Ankh'
 'lapin' 'Ankh' 'bouche' 'bouche' 'hibou' 'oeil' 'hibou' 'bouche' 'bouche'
 'Ankh' 'oeil' 'hibou' 'oeil' 'Ankh' 'hibou' 'bouche' 'Ankh' 'hibou'
 'hibou' 'hibou' 'Ankh' 'Ankh' 'hibou' 'Ankh' 'Ankh' 'Ankh' 'bouche'
 'hibou' 'lapin' 'hibou' 'hibou' 'hibou' 'lapin' 'hibou' 'lapin' 'hibou'
 'hibou' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh'
 'hibou' 'bouche' 'oeil' 'bouche' 'bouche' 'oeil' 'Ankh' 'hibou' 'hibou'
 'Ankh' 'hibou' 'hibou' 'Ankh' 'bouche' 'oeil' 'Ankh' 'hibou' 'hi

In [27]:
# train and evaluate a SVM classifer on the histogram
# representations
print("[INFO] evaluating histogram accuracy...")
for k in range(len(svm_kernel)) :
    model = SVC(kernel=svm_kernel[k], C=1, random_state=0)
    model.fit(trainFeat, trainLabels)
    Labels_pred = model.predict(testFeat)
    acc = model.score(testFeat, testLabels)
    print("[INFO] histogram accuracy: {:.2f}%".format(acc * 100))
    print(Labels_pred)

[INFO] evaluating histogram accuracy...
[INFO] histogram accuracy: 42.12%
['Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh'
 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh'
 'lapin' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh'
 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh'
 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'lapin' 'Ankh' 'Ankh'
 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh'
 'Ankh' 'Ankh' 'lapin' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh'
 'Ankh' 'Ankh' 'hibou' 'Ankh' 'Ankh' 'hibou' 'Ankh' 'Ankh' 'Ankh' 'lapin'
 'Ankh' 'Ankh' 'Ankh' 'lapin' 'lapin' 'Ankh' 'lapin' 'Ankh' 'Ankh' 'Ankh'
 'Ankh' 'Ankh' 'Ankh' 'lapin' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh'
 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh'
 'Ankh' 'lapin' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh'
 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh' 'Ankh'

In [28]:
# evaluation modèle SVM
print (metrics.classification_report(testRL, RL_pred))
print (metrics.classification_report(testLabels, Labels_pred))

             precision    recall  f1-score   support

       Ankh       0.37      1.00      0.54       122
     bouche       0.00      0.00      0.00        61
      hibou       0.00      0.00      0.00        92
      lapin       0.00      0.00      0.00        25
       oeil       0.00      0.00      0.00        30

avg / total       0.14      0.37      0.20       330

             precision    recall  f1-score   support

       Ankh       0.37      1.00      0.54       122
     bouche       0.00      0.00      0.00        61
      hibou       0.00      0.00      0.00        92
      lapin       0.00      0.00      0.00        25
       oeil       0.00      0.00      0.00        30

avg / total       0.14      0.37      0.20       330



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
